In [1]:
# import
import pandas as pd
import re
from datetime import datetime

## 내부 데이터

In [2]:
# 데이터 로드

folder = 'data/'

data_permit = pd.read_csv(folder + '건축허가현황.csv', index_col=0)
data_ordamt = pd.read_csv(folder + '국내건설수주액.csv', index_col=0)
data_unsold = pd.read_csv(folder + '미분양주택현황.csv', index_col=0)
data_rprice = pd.read_csv(folder + '아파트 실거래가격지수.csv', index_col=0)
data_mprice = pd.read_csv(folder + '유형별_주택월세통합가격지수.csv', index_col=0)
data_permit2 = pd.read_csv(folder + '주택건설인허가실적.csv', index_col=0)
data_kb1 = pd.read_csv(folder + '주택매매가격지수(KB).csv', index_col=0)
data_kb2 = pd.read_csv(folder + '주택전세가격지수(KB).csv', index_col=0)
data_cprice = pd.read_csv(folder + '지역별_지가변동률.csv', index_col=0)

datas = [data_permit, data_ordamt, data_unsold, data_rprice, data_mprice, data_permit2,
             data_kb1, data_kb2, data_cprice]
data_names = ['건축허가현황', '국내건설수주액', '미분양주택현황', '아파트 실거래가격지수', '유형별_주택월세통합가격지수',
              '주택건설인허가실적', '주택매매가격지수(KB)', '주택전세가격지수(KB)', '지역별_지가변동률']

In [23]:
# 데이터 보기

'''
for i in range(len(datas)):
    print(f'*****{data_names[i]}*****')
    display(datas[i].head())
    print(datas[i].info())
    print(datas[i].index)
    print('\n\n\n')
'''

"\nfor i in range(len(datas)):\n    print(f'*****{data_names[i]}*****')\n    display(datas[i].head())\n    print(datas[i].info())\n    print(datas[i].index)\n    print('\n\n\n')\n"

In [3]:
# 통합 파일 1 만들기 
# 지역별 주택 지표 변화 흐름을 보여주는 대시보드 구성
# 데이터 전처리

# 지역 컬럼명 정제
data_unsold.columns = [re.sub('\[.+\]', '', x) for x in data_unsold.columns]
data_rprice.columns = [re.sub('\[.+\]', '', x) for x in data_rprice.columns]
data_permit2.columns = [re.sub('\[.+\]', '', x) for x in data_permit2.columns]
data_cprice.columns = [re.sub('\[.+\]', '', x) for x in data_cprice.columns]

# 유형별_주택월세통합가격지수 유형 분리 후 지역 정제
CNUM = 24
data_mprice.columns = [re.sub('\[.+\]', '', x) for x in data_mprice.columns]
data_mprice_1 = data_mprice.iloc[:, :CNUM]  # 종합
data_mprice_2 = data_mprice.iloc[:, CNUM:CNUM*2]  # 아파트
data_mprice_3 = data_mprice.iloc[:, CNUM*2:CNUM*3]  # 연립다세대
data_mprice_4 = data_mprice.iloc[:, CNUM*3:CNUM*4]  # 단독주택

data_mprice_1.columns = [re.sub('.+-', '', x) for x in data_mprice_1.columns]
data_mprice_2.columns = [re.sub('.+-', '', x) for x in data_mprice_2.columns]
data_mprice_3.columns = [re.sub('.+-', '', x) for x in data_mprice_3.columns]
data_mprice_4.columns = [re.sub('.+-', '', x) for x in data_mprice_4.columns]

# 지역별_지가변동률 컬럼 기준으로 통합하기 위해 없는 컬럼 삭제하고 구분 컬럼 추가
data_unsold.drop(list(set(data_unsold.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_unsold['구분'] = '미분양주택현황'
data_rprice.drop(list(set(data_rprice.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_rprice['구분'] = '아파트실거래가격지수'
data_permit2.drop(list(set(data_permit2.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_permit2['구분'] = '주택건설인허가실적'
data_cprice['구분'] = '지역별지가변동률'
data_mprice_1.drop(list(set(data_mprice_1.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_mprice_1['구분'] = '종합월세통합가격지수'
data_mprice_2.drop(list(set(data_mprice_2.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_mprice_2['구분'] = '아파트월세통합가격지수'
data_mprice_3.drop(list(set(data_mprice_3.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_mprice_3['구분'] = '연립다세대월세통합가격지수'
data_mprice_4.drop(list(set(data_mprice_4.columns)-set(data_cprice.columns)), axis=1, inplace=True)
data_mprice_4['구분'] = '단독주택월세통합가격지수'

# 통합파일 생성
merge_data1 = pd.concat([data_unsold, data_rprice, data_permit2, data_cprice,
                         data_mprice_1, data_mprice_2, data_mprice_3, data_mprice_4], axis=0)
merge_data1.index.name = '날짜'
merge_data1.reset_index(inplace=True)

# 데이터 구조 변경
merge_data1 = pd.melt(merge_data1, id_vars=['날짜', '구분'], var_name='지역', value_name='수치')
merge_data1.set_index('날짜', inplace=True)
merge_data1

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-3-fa3f9662524b>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_mprice_1['구분'] = '종합월세통합가격지수'
<ipython-input-3-fa3f9662524b>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

,구분,지역,수치
날짜,,,
2007-01-01,미분양주택현황,전국,75616.0
2007-02-01,미분양주택현황,전국,73619.0
2007-03-01,미분양주택현황,전국,73162.0
2007-04-01,미분양주택현황,전국,73393.0
2007-05-01,미분양주택현황,전국,78571.0
...,...,...,...
2020-11-01,단독주택월세통합가격지수,세종,100.2
2020-12-01,단독주택월세통합가격지수,세종,100.3
2021-01-01,단독주택월세통합가격지수,세종,100.4


In [4]:
# 통합 파일 2 생성
# 주택매매가격지수 & 주택전세가격지수 통합

# 컬럼명 정제
data_kb1.columns = [re.sub('\[.+\]', '', x) for x in data_kb1.columns]
data_kb2.columns = [re.sub('\[.+\]', '', x) for x in data_kb2.columns]

# 구분 컬럼 추가
data_kb1['구분'] = '주택매매가격지수'
data_kb2['구분'] = '주택전세가격지수'

# 통합파일 생성
merge_data2 = pd.concat([data_kb1, data_kb2], axis=0)
merge_data2.index.name = '날짜'
merge_data2

,총지수,단독주택,연립주택,아파트,아파트(서울),총지수(서울),구분
날짜,,,,,,,
1986-01-01,34.656,61.347,43.053,20.973,17.153,30.044,주택매매가격지수
1986-02-01,34.656,61.347,42.925,20.973,17.183,30.044,주택매매가격지수
1986-03-01,34.708,61.513,42.733,20.935,17.153,30.002,주택매매가격지수
1986-04-01,34.449,61.097,42.733,20.819,17.034,29.837,주택매매가격지수
1986-05-01,34.293,60.848,41.837,20.664,16.945,29.588,주택매매가격지수
...,...,...,...,...,...,...,...
2020-11-01,104.886,102.823,105.011,105.140,110.216,109.252,주택전세가격지수
2020-12-01,106.043,103.249,105.711,106.608,112.406,110.887,주택전세가격지수
2021-01-01,106.925,103.545,106.306,107.688,114.116,112.229,주택전세가격지수


In [5]:
# 파일 추출
# Tableau Dashboard로 시각화 진행

merge_data1.to_csv('merge_data_1.csv', encoding='cp949')
merge_data2.to_csv('merge_data_2.csv', encoding='cp949')

## 외부 데이터
1. 통계청 '행정구역 시군구별, 성별 인구수'
2. 통계청 '시군구별 이동자수'

In [7]:
# 데이터 로드

data_pop = pd.read_csv('행정구역_시군구_별__성별_인구수.csv', encoding='cp949')
data_mv = pd.read_csv('시군구별_이동자수.csv', encoding='cp949')

In [8]:
data_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   행정구역(시군구)별  19 non-null     object
 1   2022.01     19 non-null     object
 2   2022.01.1   19 non-null     object
 3   2022.01.2   19 non-null     object
 4   2022.02     19 non-null     object
 5   2022.02.1   19 non-null     object
 6   2022.02.2   19 non-null     object
 7   2022.03     19 non-null     object
 8   2022.03.1   19 non-null     object
 9   2022.03.2   19 non-null     object
dtypes: object(10)
memory usage: 1.6+ KB


In [9]:
data_mv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   행정구역(시군구)별  19 non-null     object
 1   2022.01     19 non-null     object
 2   2022.01.1   19 non-null     object
 3   2022.01.2   19 non-null     object
 4   2022.01.3   19 non-null     object
 5   2022.01.4   19 non-null     object
 6   2022.01.5   19 non-null     object
 7   2022.01.6   19 non-null     object
 8   2022.01.7   19 non-null     object
 9   2022.02     19 non-null     object
 10  2022.02.1   19 non-null     object
 11  2022.02.2   19 non-null     object
 12  2022.02.3   19 non-null     object
 13  2022.02.4   19 non-null     object
 14  2022.02.5   19 non-null     object
 15  2022.02.6   19 non-null     object
 16  2022.02.7   19 non-null     object
 17  2022.03     19 non-null     object
 18  2022.03.1   19 non-null     object
 19  2022.03.2   19 non-null     object
 20  2022.03.3   

In [10]:
data_pop

,행정구역(시군구)별,2022.01,2022.01.1,2022.01.2,2022.02,2022.02.1,2022.02.2,2022.03,2022.03.1,2022.03.2
0,행정구역(시군구)별,총인구수 (명),남자인구수 (명),여자인구수 (명),총인구수 (명),남자인구수 (명),여자인구수 (명),총인구수 (명),남자인구수 (명),여자인구수 (명)
1,전국,51632473,25742528,25889945,51625561,25738197,25887364,51610695,25730584,25880111
2,서울특별시,9505926,4615631,4890295,9508451,4615823,4892628,9506778,4613910,4892868
3,부산광역시,3348874,1638207,1710667,3347396,1637111,1710285,3343504,1634958,1708546
4,대구광역시,2383858,1174667,1209191,2381095,1173039,1208056,2380494,1172632,1207862
5,인천광역시,2949150,1476663,1472487,2950978,1477432,1473546,2952699,1478272,1474427
6,광주광역시,1441636,713037,728599,1440032,712199,727833,1438463,711289,727174
7,대전광역시,1451272,724026,727246,1450862,723812,727050,1450057,723361,726696
8,울산광역시,1121100,575939,545161,1119446,575165,544281,1118010,574444,543566
9,세종특별자치시,374377,186907,187470,376399,187858,188541,377296,188356,188940


In [11]:
data_mv

,행정구역(시군구)별,2022.01,2022.01.1,2022.01.2,2022.01.3,2022.01.4,2022.01.5,2022.01.6,2022.01.7,2022.02,...,2022.02.6,2022.02.7,2022.03,2022.03.1,2022.03.2,2022.03.3,2022.03.4,2022.03.5,2022.03.6,2022.03.7
0,행정구역(시군구)별,총전입 (명),총전출 (명),순이동 (명),시도내이동-시군구내 (명),시도내이동-시군구간 전입 (명),시도내이동-시군구간 전출 (명),시도간전입 (명),시도간전출 (명),총전입 (명),...,시도간전입 (명),시도간전출 (명),총전입 (명),총전출 (명),순이동 (명),시도내이동-시군구내 (명),시도내이동-시군구간 전입 (명),시도내이동-시군구간 전출 (명),시도간전입 (명),시도간전출 (명)
1,전국,552315,552315,0,192710,162185,162185,197420,197420,664152,...,256975,256975,587154,587154,0,195805,169468,169468,221881,221881
2,서울특별시,100675,103083,-2408,23554,39186,39186,37935,40343,142156,...,55283,52018,122917,123192,-275,30061,46851,46851,46005,46280
3,부산광역시,38434,39047,-613,10810,17343,17343,10281,10894,44855,...,14380,14854,33217,35205,-1988,8756,14091,14091,10370,12358
4,대구광역시,23172,24337,-1165,7356,8431,8431,7385,8550,27106,...,9653,11947,26616,26248,368,6938,9275,9275,10403,10035
5,인천광역시,28815,27889,926,8912,8186,8186,11717,10791,38363,...,15843,13836,35413,33053,2360,10684,9824,9824,14905,12545
6,광주광역시,14985,14907,78,4727,4996,4996,5262,5184,18406,...,6482,7970,18246,19503,-1257,6156,6613,6613,5477,6734
7,대전광역시,20084,20938,-854,5708,6899,6899,7477,8331,22275,...,9865,10205,19200,19777,-577,5155,6334,6334,7711,8288
8,울산광역시,10509,11030,-521,3471,3476,3476,3562,4083,11693,...,4546,6187,10116,11410,-1294,3167,3348,3348,3601,4895
9,세종특별자치시,9763,7452,2311,4246,0,0,5517,3206,8610,...,5587,3718,5826,5097,729,2091,0,0,3735,3006


In [12]:
# 시도 이름 통일
# data_pop['행정구역(시군구)별'].unique()

match_dict = {'전국': '전국', '서울특별시': '서울', '부산광역시': '부산', '대구광역시': '대구', 
              '인천광역시': '인천', '광주광역시': '광주', '대전광역시': '대전', '울산광역시': '울산', 
              '경기도': '경기', '강원도': '강원', '충청북도': '충북', '충청남도': '충남', 
              '전라북도': '전북', '전라남도': '전남', '경상북도': '경북', '경상남도': '경남', 
              '제주특별자치도': '제주', '세종특별자치시': '세종'}

data_pop['행정구역(시군구)별'] = data_pop['행정구역(시군구)별'].map(match_dict)
data_mv['행정구역(시군구)별'] = data_mv['행정구역(시군구)별'].map(match_dict)
print(data_pop['행정구역(시군구)별'].unique())
print(data_mv['행정구역(시군구)별'].unique())

[nan '전국' '서울' '부산' '대구' '인천' '광주' '대전' '울산' '세종' '경기' '강원' '충북' '충남' '전북'
 '전남' '경북' '경남' '제주']
[nan '전국' '서울' '부산' '대구' '인천' '광주' '대전' '울산' '세종' '경기' '강원' '충북' '충남' '전북'
 '전남' '경북' '경남' '제주']


In [13]:
# 컬럼명 변경

data_pop.rename(columns={'시점': '날짜', '행정구역(시군구)별': '지역'}, inplace=True)
data_mv.rename(columns={'시점': '날짜', '행정구역(시군구)별': '지역'}, inplace=True)

In [14]:
# 파일 추출
# Tableau Dashboard로 시각화 진행.=> notion

data_pop.to_csv('인구수.csv', index=False, encoding='cp949')
data_mv.to_csv('인구이동.csv', index=False, encoding='cp949')